In [12]:
pip install pennylane

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import pennylane as qml
from pennylane import numpy as np

# Data loading and preprocessing
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=3)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=3)

100%|██████████| 170498071/170498071 [00:18<00:00, 9221060.05it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Files already downloaded and verified


In [4]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import pennylane as qml
from pennylane import numpy as np

# Quantum device
n_qubits = 4
q_depth = 5
q_device = qml.device('default.qubit', wires=n_qubits)

In [8]:
# Define the quantum circuit
@qml.qnode(q_device)
def q_circuit(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(w)) for w in range(n_qubits)]

# Define the hybrid model
class HybridModel(nn.Module):
    def __init__(self):
        super(HybridModel, self).__init__()
        self.densenet = models.densenet121(pretrained=True)
        self.densenet.classifier = nn.Identity()  # Remove the final layer


        # Freeze initial layers
        for param in self.densenet.parameters():
            param.requires_grad = True
        for param in self.densenet.features.denseblock4.parameters():
            param.requires_grad = True


        # Linear layer to reduce the feature size
        self.fc1 = nn.Linear(1024, n_qubits)

        # Define the TorchLayer with the quantum circuit
        weight_shapes = {"weights": (q_depth, n_qubits, 3)}
        self.q_layer = qml.qnn.TorchLayer(q_circuit, weight_shapes)
        # Dropout layer to prevent overfitting
        #self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(n_qubits, 10)  # Final classification layer


    def forward(self, x):
        x = self.densenet(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)  # Project to match the number of qubits
        #x = self.dropout(x)  # Apply dropout after projection to qubit size
        x = self.q_layer(x)
        x = self.fc2(x)
        return x





In [9]:
# Instantiate the model, loss function, and optimizer
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = HybridModel().to(device)
print(model)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


HybridModel(
  (densenet): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momentu

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)# Learning rate finder

#optimizer = optim.AdamW(model.parameters(), lr=0.001)  # Use weight decay


# Learning rate scheduler
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [11]:
# Training and evaluation loop
num_epochs = 20
device = 'cuda' if torch.cuda.is_available() else 'cpu'

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        optimizer.zero_grad()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100:.4f}, Accuracy: {100 * correct / total:.2f}%')
            running_loss = 0.0

        # Adjust the learning rate
    #scheduler.step()


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 1, Batch 100, Loss: 2.2261, Accuracy: 17.25%
Epoch 1, Batch 200, Loss: 2.0907, Accuracy: 21.68%
Epoch 1, Batch 300, Loss: 1.9785, Accuracy: 26.33%
Epoch 2, Batch 100, Loss: 1.8019, Accuracy: 39.34%
Epoch 2, Batch 200, Loss: 1.7636, Accuracy: 39.64%
Epoch 2, Batch 300, Loss: 1.7249, Accuracy: 39.52%
Epoch 3, Batch 100, Loss: 1.6336, Accuracy: 43.00%
Epoch 3, Batch 200, Loss: 1.5997, Accuracy: 43.87%
Epoch 3, Batch 300, Loss: 1.5639, Accuracy: 44.35%
Epoch 4, Batch 100, Loss: 1.5049, Accuracy: 49.16%
Epoch 4, Batch 200, Loss: 1.5062, Accuracy: 49.14%
Epoch 4, Batch 300, Loss: 1.4935, Accuracy: 49.38%
Epoch 5, Batch 100, Loss: 1.4487, Accuracy: 51.48%
Epoch 5, Batch 200, Loss: 1.4400, Accuracy: 51.66%
Epoch 5, Batch 300, Loss: 1.4128, Accuracy: 52.06%
Epoch 6, Batch 100, Loss: 1.3987, Accuracy: 53.58%
Epoch 6, Batch 200, Loss: 1.3669, Accuracy: 54.06%
Epoch 6, Batch 300, Loss: 1.3790, Accuracy: 54.01%
Epoch 7, Batch 100, Loss: 1.3515, Accuracy: 54.66%
Epoch 7, Batch 200, Loss: 1.335

KeyboardInterrupt: 

In [ ]:
# Separate evaluation on the test set
model.eval()
correct = 0
total = 0
test_loss = 0.0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
test_loss /= len(testloader)

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

Test Loss: 0.7022, Test Accuracy: 78.60%
